In [1]:
!pip install WordCloud

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import zipfile
import os


In [2]:
df = pd.read_csv('feedback_final.csv')
df = df[['text', 'rating', 'mood']]
df.mood = df.mood.replace(22, 2)
df['mood'] = df['mood'].replace({0: 'negative', 1: 'neutral', 2: 'positive'})
df.head()


,text,rating,mood
0,"Вообщем комод за такую цену норм , всех детале...",5,positive
1,"Комод просто замечательный,очень легко и удоб...",5,positive
2,К сожалению разочарование. Во первых не положи...,1,negative
3,"За свои деньги хороший товар. Собрали быстро, ...",5,positive
4,"Комод хороший мне понравился , пришёл целый , ...",5,positive


In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 24.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [5]:
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
from datasets import Dataset
from scipy.special import softmax


In [6]:
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# Разделяем данные на train (70%), validation (20%) и test (10%)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.333, random_state=42)  # 0.333 * 0.3 ≈ 0.1

# Создаем наборы данных для использования с Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


In [8]:
# Функция токенизации с учетом максимальной длины
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)


In [9]:
# Токенизация данных
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6073 [00:00<?, ? examples/s]

Map:   0%|          | 0/1736 [00:00<?, ? examples/s]

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

In [10]:
# Преобразование меток в числовой формат
label_mapping = {"positive": 2, "neutral": 1, "negative": 0}

def encode_labels(examples):
    examples['label'] = [label_mapping[mood] for mood in examples['mood']]
    return examples

train_dataset = train_dataset.map(encode_labels, batched=True)
val_dataset = val_dataset.map(encode_labels, batched=True)
test_dataset = test_dataset.map(encode_labels, batched=True)

Map:   0%|          | 0/6073 [00:00<?, ? examples/s]

Map:   0%|          | 0/1736 [00:00<?, ? examples/s]

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

In [11]:
# Удаление колонок, которые не нужны для обучения
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

# Удаление ненужной колонки
train_dataset = train_dataset.remove_columns(["mood"])
val_dataset = val_dataset.remove_columns(["mood"])
test_dataset = test_dataset.remove_columns(["mood"])


In [16]:
!pip install accelerate -U

In [14]:
!pip install transformers[torch]

In [30]:
!pip install --upgrade accelerate
!pip install transformers[torch]
!pip install --upgrade transformers


In [12]:
import accelerate
import transformers

print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)


Accelerate version: 0.32.1
Transformers version: 4.42.4


In [35]:
!pip install torch --upgrade
!pip uninstall transformers accelerate -y
!pip install transformers[torch]
!pip install accelerate -U


Found existing installation: transformers 4.42.4
Uninstalling transformers-4.42.4:
  Successfully uninstalled transformers-4.42.4
Found existing installation: accelerate 0.32.1
Uninstalling accelerate-0.32.1:
  Successfully uninstalled accelerate-0.32.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 43.1 MB/s eta 0:00:00
  Using cached accelerate-0.32.1-py3-none-any.whl (314 kB)


In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Загрузка модели
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

# Определение аргументов для обучения
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Используйте eval_strategy вместо evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [14]:
# Определение data collator
data_collator = DataCollatorWithPadding(tokenizer)

In [15]:
# Функции для вычисления метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [16]:
# Определение Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [18]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Обучение модели
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.560920,0.793203,0.767176,0.743660,0.793203
2,0.537700,0.523959,0.808756,0.788017,0.778890,0.808756
3,0.445000,0.529264,0.817396,0.796947,0.791899,0.817396


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1140, training_loss=0.4824066262496145, metrics={'train_runtime': 505.894, 'train_samples_per_second': 36.013, 'train_steps_per_second': 2.253, 'total_flos': 1597887785862144.0, 'train_loss': 0.4824066262496145, 'epoch': 3.0})

In [19]:
test_results = trainer.evaluate(test_dataset)
test_results

{'eval_loss': 0.47519558668136597,
 'eval_accuracy': 0.8350634371395617,
 'eval_f1': 0.8151283288512349,
 'eval_precision': 0.8014398981646276,
 'eval_recall': 0.8350634371395617,
 'eval_runtime': 6.4833,
 'eval_samples_per_second': 133.729,
 'eval_steps_per_second': 8.483,
 'epoch': 3.0}